In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0009_poisson_2d"
config.target = "imprs_2022"
config.debug_mode = True

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams.update(config.tueplots_bundle())

In [ ]:
rng = np.random.default_rng(24)

## Problem Definition

In [ ]:
domain= linpde_gp.domains.Box([[-1.0, 1.0], [-1.0, 1.0]])

In [ ]:
rhs = linpde_gp.functions.Constant(input_shape=(2,), value=2.0)

In [ ]:
boundary_function = linpde_gp.functions.Constant(input_shape=(2,), value=0.0)

In [ ]:
bvp = linpde_gp.problems.pde.PoissonEquationDirichletProblem(domain, rhs, boundary_function)

In [ ]:
plt_grid_t, plt_grid_x = np.meshgrid(
    np.linspace(*domain[0], 50),
    np.linspace(*domain[1], 50),
    indexing="ij",
)

plt_grid = np.stack((plt_grid_t, plt_grid_x), axis=-1)

## Prior

In [ ]:
prior_gp = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Zero(input_shape=(2,)),
    cov=2.0 ** 2 * linpde_gp.randprocs.kernels.ExpQuad(
        input_shape=(2,),
        lengthscales=0.5,
    ),
)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid_t, plt_grid_x, prior_gp.mean(plt_grid))

## Boundary Conditions

In [ ]:
N_bc = 9

bc_Xia = np.stack(
    (
        np.linspace(*domain[0], N_bc),
        np.full(N_bc, domain[1][0])
    ),
    axis=-1
)

bc_Xib = np.stack(
    (
        np.linspace(*domain[0], N_bc),
        np.full(N_bc, domain[1][1])
    ),
    axis=-1
)

bc_Xai = np.stack(
    (
        np.full(N_bc, domain[0][0])[1:-1],
        np.linspace(*domain[1], N_bc)[1:-1]
    ),
    axis=-1
)

bc_Xbi = np.stack(
    (
        np.full(N_bc, domain[0][1])[1:-1],
        np.linspace(*domain[1], N_bc)[1:-1],
    ),
    axis=-1
)

bc_X = np.concatenate(
    (bc_Xia, bc_Xib, bc_Xai, bc_Xbi),
    axis=0,
)

In [ ]:
u_bc = prior_gp.condition_on_observations(
    bc_X,
    bvp.boundary_conditions[0].values(bc_X),
)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid_t, plt_grid_x, u_bc.mean(plt_grid))

### Predictive Induced by $\Delta$

In [ ]:
Lu_bc = bvp.diffop(u_bc)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid_t, plt_grid_x, Lu_bc.mean(plt_grid))

### Conditioning on the PDE

In [ ]:
N_pde = 7
eps = 0.0

pde_X = np.stack(
    np.meshgrid(
        np.linspace(domain[0][0] + eps, domain[0][1] + eps, N_pde),
        np.linspace(domain[1][0] + eps, domain[1][1] + eps, N_pde),
        indexing="ij",
    ),
    axis=-1,
).reshape(-1, 2)

u_bc_pde = u_bc.condition_on_observations(
    X=pde_X,
    Y=bvp.rhs(pde_X),
    L=bvp.diffop,
)

In [ ]:
# %matplotlib widget

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid_t, plt_grid_x, u_bc_pde.mean(plt_grid))
ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")

In [ ]:
# %matplotlib inline

In [ ]:
plt.figure()

X_std = np.reshape(plt_grid, (-1, 2))
u_post_std = 1.95 * u_bc_pde.std(X_std).reshape(plt_grid.shape[:-1])

im = plt.imshow(u_post_std, cmap="coolwarm")
plt.colorbar()

In [ ]:
from mpl_toolkits import axes_grid1

fig = plt.figure()
spec = fig.add_gridspec(ncols=2, width_ratios=[0.55, 0.45])

fig.set_constrained_layout_pads(w_pad=10 / 72, h_pad=4 / 72)

# Mean
ax_mean = fig.add_subplot(spec[0], projection="3d")
u_post_mean_surf = ax_mean.plot_surface(
    plt_grid_t,
    plt_grid_x,
    u_bc_pde.mean(plt_grid),
)

ax_mean.set_title("Posterior Mean")

# 95% Credible Interval
ax_std = fig.add_subplot(spec[1])
u_post_std_img = ax_std.imshow(
    u_post_std,
    cmap="coolwarm",
    extent=[-1., 1., -1., 1.],
)

ax_std.set_title("95% Confidence Interval")

# Colorbar
divider = axes_grid1.make_axes_locatable(ax_std)
cax = divider.append_axes(
    "right",
    size="5%",
    pad=plt.rcParams['figure.subplot.wspace'] / 3.0
)
fig.colorbar(u_post_std_img, cax=cax)

experiment_utils.savefig("poisson_2d_posterior")

In [ ]:
fig, ax = plt.subplots(ncols=2)

# Mean
u_post_mean_img = ax[0].imshow(
    u_bc_pde.mean(plt_grid),
    cmap="coolwarm",
    extent=[-1., 1., -1., 1.],
)

divider = axes_grid1.make_axes_locatable(ax[0])
cax = divider.append_axes(
    "right",
    size="5%",
    pad=plt.rcParams['figure.subplot.wspace'] / 4.0
)
fig.colorbar(u_post_mean_img, cax=cax)

ax[0].set_title("Posterior Mean")

# 95% Credible Interval
u_post_std_img = ax[1].imshow(
    u_post_std,
    cmap="coolwarm",
    extent=[-1., 1., -1., 1.],
)

divider = axes_grid1.make_axes_locatable(ax[1])
cax = divider.append_axes(
    "right",
    size="5%",
    pad=plt.rcParams['figure.subplot.wspace'] / 3.0
)
fig.colorbar(u_post_std_img, cax=cax)

ax[1].set_title("95% Confidence Interval")

experiment_utils.savefig("poisson_2d_posterior_heatmap")